### Snapping

In this notebook, we take our origin points (here, derived from a WorldPop raster manually using the 'Raster pixels to Points' tool, and saved as as shapefile. 

At the end of this notebook, we are ready to generate the OD Matrix

In [1]:
import os, sys
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, MultiPoint
from shapely.wkt import loads
from scipy import spatial
from functools import partial
import pyproj
from shapely.ops import transform
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn

networkx version: 2.3 
osmnx version: 0.9 


Import Graph

In [2]:
net_path = r'C:\Users\charl\Documents\GOST\SierraLeone\RoadNet'
net_file = r'final_G.pickle'
G = nx.read_gpickle(os.path.join(net_path, net_file))

Import Origins

In [3]:
origin_folder = r'C:\Users\charl\Documents\GOST\SierraLeone\Origins'
origin_file = r'origins_100m.shp'
origins = gpd.read_file(os.path.join(origin_folder, origin_file))

Snap Origins

In [ ]:
origins_snapped = gn.pandana_snap(G, 
                                  origins, 
                                  source_crs='epsg:4326',
                                  target_crs='epsg:3857', 
                                  add_dist_to_node_col = True)

C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets\GOSTnet.py:1645: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())
C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets\GOSTnet.py:1647: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(in_df[['x','y']].as_matrix())


Save Origins to origin folder

In [ ]:
origins_snapped.to_csv(os.path.join(origin_folder, origin_file.replace('.shp','_snapped.csv')))

Import Destinations, convert Long and Lat columns over to shapely point geometries

In [ ]:
dest_path = r'C:\Users\charl\Documents\GOST\SierraLeone\Destinations'
school_fil = r'schools.csv'
health_fil = r'health_centers.csv'

df_schools = pd.read_csv(os.path.join(dest_path, school_fil))
df_schools['geometry'] = list(zip(df_schools['LONGITUDE'],df_schools['LATITUDE']))
df_schools['geometry'] = df_schools['geometry'].apply(Point)
gdf_schools = gpd.GeoDataFrame(df_schools, crs = {'init':'epsg:4326'}, geometry = 'geometry')

df_health = pd.read_csv(os.path.join(dest_path, health_fil))
df_health['geometry'] = list(zip(df_health['LONGITUDE'].astype(float),df_health['LATITUDE'].astype(float)))
df_health['geometry'] = df_health['geometry'].apply(Point)
gdf_health = gpd.GeoDataFrame(df_health, crs = {'init':'epsg:4326'}, geometry = 'geometry')

Snap Destinations. Pay special attention to the crs - need to project in order to get a useful distnace to nearest node column!

In [ ]:
df_schools_snapped = gn.pandana_snap(G, 
                                     gdf_schools, 
                                     source_crs='epsg:4326',
                                     target_crs='epsg:3857', 
                                     add_dist_to_node_col = True)
df_health_snapped  = gn.pandana_snap(G, 
                                     gdf_health, 
                                     source_crs='epsg:4326',
                                     target_crs='epsg:3857', 
                                     add_dist_to_node_col = True)

Save Destinations to destinations folder

In [ ]:
df_schools_snapped.to_csv(os.path.join(dest_path, school_fil.replace('.csv','_snapped.csv')))
df_health_snapped.to_csv(os.path.join(dest_path, health_fil.replace('.csv','_snapped.csv')))

Send copies to the graphtool folder for processing

In [ ]:
gtool = r'C:\Users\charl\Documents\GOST\SierraLeone\graphtool'

origins_snapped.to_csv(os.path.join(gtool, origin_file.replace('.shp','_snapped.csv')))

df_schools_snapped.to_csv(os.path.join(gtool, school_fil.replace('.csv','_snapped.csv')))

df_health_snapped.to_csv(os.path.join(gtool, health_fil.replace('.csv','_snapped.csv')))